# BAT.jl: Background + Signal Analysis 

In [ ]:
using Random, LinearAlgebra, Statistics, Distributions, StatsBase
using EponymTuples
using BAT, IntervalSets
using ValueShapes
using Plots
pyplot()

## Generate data:

In [ ]:
bins=0:0.5:30
N_background = 1500
λ = 10

data_background = rand(Truncated(Exponential(λ), 0, 30), N_background)
hist_background = append!(Histogram(bins), data_background);

In [ ]:
N_signal = 46
N_s_background = 400
μ_true = 15.0
σ_true = 0.7

data_signal =  vcat(
    rand(Truncated(Exponential(λ), 0, 30), N_s_background),
    rand(Normal( μ_true, σ_true), N_signal)
)

hist_signal = append!(Histogram(bins), data_signal);

In [ ]:
plot(normalize(hist_signal, mode=:density), alpha=0.5, color=:red, label="Signal Data")
plot!(normalize(hist_background, mode=:density),  seriestype = :steps, lw=2.5, alpha=0.9, color=:black, label="Background Data")
plot!(xlabel = "x", ylabel="P(x)", title="Data for Analysis")


## Model functions: 

In [ ]:
function background_model(@eponymargs(A, λ), x::Real)
    A[1] + A[2]*exp(-λ[1]*x)
end

function background_signal_model(@eponymargs(A, λ, D, x_0, G), x::Real)
    A[1] + A[2]*exp(-λ[1]*x) + (D[1]/(sqrt(2*pi*G[1]^2)))*exp(-((x - x_0[1])^2)/G[1]^2)  
end

## Likelihood function with Poisson noise: 

In [ ]:
struct HistogramLikelihood{H<:Histogram,F<:Function} <: AbstractDensity
    histogram::H
    fitfunc::F
end

In [ ]:
function BAT.density_logval(
    likelihood::HistogramLikelihood,
    params::Union{NamedTuple,AbstractVector{<:Real}}
)
    # Histogram counts for each bin as an array:
    counts = likelihood.histogram.weights

    # Histogram binning, has length (length(counts) + 1):
    binning = likelihood.histogram.edges[1]
    
    log_likelihood::Float64 = 0.0
    for i in eachindex(counts)
        bin_left, bin_right = binning[i], binning[i+1]
        bin_width = bin_right - bin_left
        bin_center = (bin_right + bin_left) / 2
        observed_counts = counts[i]
        expected_counts = bin_width * likelihood.fitfunc(params, bin_center)
        if expected_counts > 0 
            log_likelihood += logpdf(Poisson(expected_counts), observed_counts)
        else
           log_likelihood += -Inf
        end 
    end

    return log_likelihood
end

In [ ]:
algorithm = MetropolisHastings(MvTDistProposalSpec(1.0))
rngseed = BAT.Philox4xSeed()
nsamples = 5*10^5
max_nsteps = 5*10^5
nchains = 8

tuner_config = ProposalCovTunerConfig(
    λ = 0.5,
    α = 0.15..0.35,
    β = 1.5,
    c = 1e-4..1e2
)

convergence_test = BGConvergence(
    threshold = 1.1,
    corrected = false
)

init_strategy = MCMCInitStrategy(
    ninit_tries_per_chain = 8..128,
    max_nsamples_pretune = 25,
    max_nsteps_pretune = 250,
    max_time_pretune = Inf
)

burnin_strategy = MCMCBurninStrategy(
    max_nsamples_per_cycle = 10000,
    max_nsteps_per_cycle = 20000,
    max_time_per_cycle = Inf,
    max_ncycles = 100
);

ENV["JULIA_INFO"] = "BAT"

## Background analysis: 

In [ ]:
likelihood_bm = HistogramLikelihood(hist_background, background_model)

In [ ]:
prior_bm = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = [0.0 .. 20.0]
);


In [ ]:
parshapes_bm = VarShapes(prior_bm)
posterior_bm = PosteriorDensity(likelihood_bm, prior_bm)
chainspec_bm = MCMCSpec(algorithm, posterior_bm, rngseed);

In [ ]:
samples_bm, sampleids_bm, stats_bm, chains_bm = rand(
    chainspec_bm,
    nsamples,
    nchains,
    tuner_config = tuner_config,
    convergence_test = convergence_test,
    init_strategy = init_strategy,
    burnin_strategy = burnin_strategy,
    max_nsteps = max_nsteps,
    max_time = Inf,
    granularity = 1
);

In [ ]:
println("Mode: $(stats_bm.mode)")
println("Mean: $(stats_bm.param_stats.mean)")

In [ ]:
mode_parms_bm = parshapes_bm(stats_bm.mode)

In [ ]:
plot(samples_bm, params=[1,2,3])

In [ ]:
plot(normalize(hist_background, mode=:density),  seriestype = :steps, lw=2.5, alpha=0.9, color=:black, label="Background Data")
plot!(xlabel = "x", ylabel="P(x)", title="Background Analysis")

plot!(
    bins, x -> background_model(mode_parms_bm, x),
    label = "Background Model", lw=1.5, color=1
)


### Histogram as a prior: 

In [ ]:
samples = hcat(samples_bm.params...);

λ_min, λ_max = minimum(samples[3,:]), maximum(samples[3,:])
δ = (λ_max-λ_min)/150
λ_hist = Histogram(λ_min:δ:λ_max)

append!(λ_hist, samples[3,:])

## Background + signal analysis: 

In [ ]:
likelihood_signal = HistogramLikelihood(hist_signal, background_signal_model);

prior_signal = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = [BAT.HistogramAsUvDistribution(λ_hist)],
    D = [0.0..1000], 
    x_0 = [0 .. 30], 
    G = [0.0 .. 5.0]
);


parshapes_signal = VarShapes(prior_signal)
posterior_signal = PosteriorDensity(likelihood_signal, prior_signal);
chainspec_signal = MCMCSpec(algorithm, posterior_signal, rngseed);

In [ ]:
samples_signal, sampleids_signal, stats_signal, chains_signal = rand(
    chainspec_signal,
    nsamples,
    nchains,
    tuner_config = tuner_config,
    convergence_test = convergence_test,
    init_strategy = init_strategy,
    burnin_strategy = burnin_strategy,
    max_nsteps = max_nsteps,
    max_time = Inf,
    granularity = 1
);

In [ ]:
plot(samples_signal, params=[1,2,3])

In [ ]:
plot(samples_signal, params=[4,5,6])

In [ ]:
plot(samples_signal, 3)
plot!(prior_signal, 3)
plot!(ylabel="P(λ)", xlabel="λ", title="Knowledge Update")

In [ ]:
mode_parms_signal = parshapes_signal(stats_signal.mode)

In [ ]:
plot(normalize(hist_signal, mode=:density), alpha=0.5, color=:red, label="Signal Data")

plot!(xlabel = "x", ylabel="P(x)")

plot!(
    bins, x -> background_signal_model(mode_parms_signal, x),
    label = "Signal Model", lw=2, color=6
)



In [ ]:
Z_signal = HMIData(samples_signal)

hm_integrate!(Z_signal)

## Signal vs. No signal: 

In [ ]:
likelihood_nosignal = HistogramLikelihood(hist_signal, background_model);

prior_nosignal = NamedPrior(
    A = [0.0 .. 1000.0, 0 .. 10000.0],
    λ = [BAT.HistogramAsUvDistribution(λ_hist)],
);


parshapes_nosignal = VarShapes(prior_nosignal)
posterior_nosignal = PosteriorDensity(likelihood_nosignal, prior_nosignal);
chainspec_nosignal = MCMCSpec(algorithm, posterior_nosignal, rngseed);

In [ ]:
samples_nosignal, sampleids_nosignal, stats_nosignal, chains_nosignal = rand(
    chainspec_nosignal,
    nsamples,
    nchains,
    tuner_config = tuner_config,
    convergence_test = convergence_test,
    init_strategy = init_strategy,
    burnin_strategy = burnin_strategy,
    max_nsteps = max_nsteps,
    max_time = Inf,
    granularity = 1
);

In [ ]:
mode_parms_nosignal = parshapes_nosignal(stats_nosignal.mode)

In [ ]:
plot(normalize(hist_signal, mode=:density), alpha=0.5, color=:red, label="Signal Data")
plot!(xlabel = "x", ylabel="P(x)")

plot!(
    bins, x -> background_model(mode_parms_nosignal, x),
    label = "No signal assumption", lw=2, color=3
)

plot!(
    bins, x -> background_signal_model(mode_parms_signal, x),
    label = "Signal assumption", lw=2, color=6
)



In [ ]:
Z_nosignal = HMIData(samples_nosignal)
        
hm_integrate!(Z_nosignal)

In [ ]:
Z_signal_v = Z_signal.integralestimates["analytic result"].final.estimate
Z_nosignal_v = Z_nosignal.integralestimates["analytic result"].final.estimate;

@show Z_signal_v/Z_nosignal_v;